## Download Twitter Data

<!--
import data_analytics.github as github
print(github.create_jupyter_notebook_header("tahirawwad", "agriculture-data-analytics", "notebooks/notebook-1-04-dc-twitter-api.ipynb", "master"))
-->
<table style="margin: auto;"><tr><td><a href="https://mybinder.org/v2/gh/tahirawwad/agriculture-data-analytics/master?filepath=notebooks/notebook-1-04-dc-twitter-api.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a></td><td>online editors</td><td><a href="https://colab.research.google.com/github/tahirawwad/agriculture-data-analytics/blob/master/notebooks/notebook-1-04-dc-twitter-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></td></tr></table>

### Setup
Import required third party Python libraries, import supporting functions and sets up data source file paths.

In [1]:
# Local
#!pip install -r script/requirements.txt
# Remote option
#!pip install -r https://raw.githubusercontent.com/tahirawwad/agriculture-data-analytics/requirements.txt
#Options: --quiet --user

In [2]:
from configparser import ConfigParser
from pandas import DataFrame
import csv
import pandas
import tweepy

### Twitter
#### Irish Farming Accounts

Dept of Agriculture, Food and the Marine:  <a href="https://twitter.com/agriculture_ie" target="_new">@agriculture_ie</a>  
Leading the sustainable development of Ireland’s agri-food, forestry and marine sectors.  

Irish Farmers' Association: <a href="https://twitter.com/ifamedia" target="_new">@ifamedia</a>  
IFA has protected and defended the interests of Irish farmers and their families at home and in Europe for more than 65 years.  

Irish Farmers Journal : <a href="https://twitter.com/farmersjournal" target="_new">@farmersjournal</a>  
Ireland's largest selling farming & rural living publication | Readership: 321,400 weekly print readers on the Island of Ireland. Source: Kantar 2021 TGI Survey

<a href="https://twitter.com/teagasc" target="_new">@Teagasc</a>  
Teagasc - Providing research, advisory & training services to the agriculture and food industry & rural communities.

<a href="https://twitter.com/thatsfarming" target="_new">@thatsfarming.com</a>  
Latest news on all things Farming. If you send us a tweet, you consent to letting http://Thatsfarming.com use & showcase it in any media.

<a href="https://twitter.com/agrilandireland" target="_new">@AgrilandIreland</a>  
Ireland’s largest farming news portal. Daily updates covering all agricultural sectors.

#### US and World Accounts

<a href="https://twitter.com/USDA" target="_new">@USDA</a> (131,000+ followers). Stay up-to-date with the latest news and information from the USDA, including links to the USDA Blog, recent videos and commentary from Secretary of Agriculture Tom Vilsack. @USDA also live tweets from industry events.

<a href="https://twitter.com/MonsantoCo" target="_new">@MonsantoCo</a> (24,000+). Monsanto provides regular updates on the company’s latest innovation and news, as well as access to the company’s blog, Beyond the Rows. The blog is updated by Monsanto employees who write about the company’s business, the ag industry, the farmer and more.

<a href="https://twitter.com/nationalffa" target="_new">@nationalffa</a> (24,000+). Formerly the Future Farmers of America, the National FFA Organization uses Twitter as a great way to keep its followers (and members) current on activities such as awards programs, outreach efforts and career opportunities.

<a href="https://twitter.com/farmbureau" target="_new">@farmbureau</a> (18,000+). American Farm Bureau is the nation’s largest farm organization, comprised of and directed by farm and ranch families who engage in all types of food and fiber production. @farmbureau gives followers access to its newsletter (Foodie News) and podcast (FoodieCast), which offer the latest in food trends.

<a href="https://twitter.com/farmingfirst" target="_new">@farmingfirst</a> (16,500+). Farming First is a global coalition that calls on world leaders to increase agricultural output in a sustainable and socially responsible manner. Many tweets include links to content found on FarmingFirst.org, which features blogs, case studies, video and more.

<a href="https://twitter.com/rodaleinstitute" target="_new">@rodaleinstitute</a> (15,500+). A leader in the organics industry since 1947, Rodale Institute has a strong following on Twitter. The institute shares its latest research on the best practices of organic agriculture with farmers and scientists throughout the world.

In [3]:
config_filepath = "config.ini"
config_parser = ConfigParser()
config_parser.read(config_filepath)

access_token = config_parser["Twitter"]["AccessToken"]
access_token_secret = config_parser["Twitter"]["AccessTokenSecret"]
consumer_key = config_parser["Twitter"]["ApiKey"]
consumer_secret = config_parser["Twitter"]["ApiKeySecret"]

<https://python-twitter.readthedocs.io/en/latest/getting_started.html>  


You need to have a developer account: <https://developer.twitter.com/en/portal/petition/essential/basic-info>

And apply for elevated access.
<https://developer.twitter.com/en/portal/products/elevated>

In [4]:
o_auth_handler = tweepy.OAuthHandler(consumer_key, consumer_secret)
o_auth_handler.set_access_token(access_token, access_token_secret)
tweepy_api = tweepy.API(o_auth_handler, wait_on_rate_limit=True)

In [5]:
#Dept of Agriculture, Food and the Marine
screen_name = "agriculture_ie"

In [6]:
tweets = tweepy_api.user_timeline(
    screen_name=screen_name,
    count=200,  # 200 is the maximum allowed count
    include_rts=False,
    tweet_mode="extended"
)  # Necessary to keep full_text otherwise only the first 140 words are extracted

In [7]:
for info in tweets[:3]:
    print("ID: {}".format(info.id))
    print(info.created_at)
    print(info.full_text)
    print("\n")

ID: 1486384591313293320
2022-01-26 17:04:36+00:00
Reopening the scheme,Minister @pippa_hackett said with 'expanding market opportunities for Irish organic produce and the drive for Irish farming to be more environmentally friendly, now is surely the time for all Irish farmers to give serious consideration to joining the scheme.'


ID: 1486383042587418633
2022-01-26 16:58:27+00:00
Farmers could qualify for yearly payments of up to €220/ha during the conversion period and up to €170/ha for full organic status.

Higher rates are available for organic #horticultural and #tillage farmers

For a guide to the application process, visit: https://t.co/wPRP91ZYJU


ID: 1486383040310005765
2022-01-26 16:58:27+00:00
New applicants interested in converting to #organic farming can apply to our Organic Farming Scheme when it reopens on Wednesday, February 9 with the budget increased by a further €5 million for new entrants
🗓️Farmers have until Friday, April 8 to apply
📍https://t.co/Chd5TGz9uZ https:/

In [8]:
#extract additional tweets

all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id
while True:
    tweets = tweepy_api.user_timeline(
        screen_name=screen_name,
        count=200,# 200 is the maximum allowed count
        include_rts=False,
        max_id=oldest_id - 1,
        # Necessary to keep full_text
        # otherwise only the first 140 words are extracted
        tweet_mode='extended')
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

N of tweets downloaded till now 249
N of tweets downloaded till now 383
N of tweets downloaded till now 534
N of tweets downloaded till now 676
N of tweets downloaded till now 832
N of tweets downloaded till now 992
N of tweets downloaded till now 1170
N of tweets downloaded till now 1340
N of tweets downloaded till now 1478
N of tweets downloaded till now 1594
N of tweets downloaded till now 1715
N of tweets downloaded till now 1827
N of tweets downloaded till now 1934
N of tweets downloaded till now 2058
N of tweets downloaded till now 2172
N of tweets downloaded till now 2205


In [9]:
tweets_list: list = [[
    tweet.id_str, tweet.user.screen_name, tweet.created_at,
    tweet.favorite_count, tweet.retweet_count,
    tweet.full_text.encode("utf-8").decode("utf-8")
] for idx, tweet in enumerate(all_tweets)]

In [10]:
tweet_columns = [
    "id", "screen_name", "created_at", "favorite_count", "retweet_count",
    "text"
]
dataframe = DataFrame(tweets_list, columns=tweet_columns)
dataframe.to_csv('./../assets/twitter-agriculture-ie.csv', index=False)
dataframe.head(3)

id     screen_name                created_at  \
0  1486384591313293320  agriculture_ie 2022-01-26 17:04:36+00:00   
1  1486383042587418633  agriculture_ie 2022-01-26 16:58:27+00:00   
2  1486383040310005765  agriculture_ie 2022-01-26 16:58:27+00:00   

   favorite_count  retweet_count  \
0               5              5   
1               3              3   
2              24             30   

                                                text  
0  Reopening the scheme,Minister @pippa_hackett s...  
1  Farmers could qualify for yearly payments of u...  
2  New applicants interested in converting to #or...